# A0149963M
## Assignment 4
## YSC2229 - Introductory Data Structures and Algorithms

Solution for Problem 1 - Runway Scheduling System

**1(a) - Store an approved/valid request in O(log n), where the stored information includes: aircraft ID (with format "AA123456") and take-off time**

This question has many parts. For now, I wil define the following:

In [36]:
Red = 'Red'
Black = 'Black'

In [37]:
class RBNode():
    def __init__(self, id_no = None, time = None):
        self.time = time
        self.id_no = id_no
        self.p = None
        self.left = None
        self.right = None
        self.col = Red 

In [64]:
class RedBlackTree():
    def __init__(self):
        self.NIL = RBNode()
        self.NIL.col = Black
        self.NIL.left = None
        self.NIL.right = None
        self.root = self.NIL
    
    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.left:
            x.p.left = y
        else:
            x.p.right = y
        y.left = x
        x.p = y
    
    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.right:
            x.p.right = y
        else: 
            x.p.left = y
        y.right = x
        x.p = y
    
        
    def fix_insert(self, z):
        while z.p.col == Red:
            if z.p == z.p.p.left:
                y = z.p.p.right
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.right:
                        z = z.p
                        self.left_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.right_rotate(z.p.p)
            else:
                y = z.p.p.left
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.left:
                        z = z.p
                        self.right_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.left_rotate(z.p.p)
            
        self.root.col = Black
      
        
    def insert(self, id_key, time_key):
        z = RBNode(id_key, time_key)
        
        y = self.NIL
        x = self.root
        
        while x != self.NIL:
            y = x
            if z.time < x.time:
                x = x.left
            else: 
                x = x.right
            
        z.p = y
        if y == self.NIL:
            self.root = z
        elif z.time < y.time:
            y.left = z
        else: 
            y.right = z
        
        z.left = self.NIL
        z.right = self.NIL
        z.col = Red
        
        self.fix_insert(z)
        
        
    def find_min(self):
        node = self.root
        while node.left != self.NIL:
            node = node.left
        print("Minimum time is:", node.time)
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def find_max(self):
        node = self.root
        while node.right != self.NIL:
            node = node.right
        print("Maximum time is:", node.time)
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def print_node(self, x):
        print("ID: ", x.id_no, ", Time: ", x.time)
        
    
    def inorder(self, node):
        if node != self.NIL:
            self.inorder(node.left)
            self.print_node(node)
            self.inorder(node.right)
    
    def search(self):
        pass

Testing functions

In [63]:
t = RedBlackTree()
t.insert("AR190237", 13)
t.insert("AS123984", 21)
t.insert("AE123973", 19)
t.insert("AR812731", 15)
t.insert("AS239183", 14)

t.find_min()
t.find_max()

t.inorder(t.root)


Minimum time is: 13
Corresponding ID number is: AR190237 

Maximum time is: 21
Corresponding ID number is: AS123984 

ID:  AR190237 , Time:  13
ID:  AS239183 , Time:  14
ID:  AR812731 , Time:  15
ID:  AE123973 , Time:  19
ID:  AS123984 , Time:  21


Solution for Problem 2 - Belief Propogation

Solution for Problem 3 - Huffman Code

In [57]:
class Queue:
    def __init__(self, size):
        self.items = [None for i in range(size)]
        self.tail = 0
        self.head = 0
        self.cache = 0
        self.size = size
    
    def enqueue(self, data):
        self.cache += 1
        if self.cache > self.size:
            print("overflow")
            return
        self.items[self.tail] = data
        self.tail = (self.tail + 1) % self.size
        
        
    def dequeue(self):
        self.cache -= 1
        if self.cache < 0:
            print("underflow")
            return
        data = self.items[self.head]
        self.head = (self.head + 1) % self.size
        return data
    

In [62]:
def char_count(s):
    seen = {}
    for c in s:
        if c not in seen:
            seen[c] = 1
        else:
            seen[c] += 1
    return seen


def huffman_code(arr):
    """
    arr - array of sentences
    """
    for sentence in arr:
        temp = char_count(sentence)
        arr = []
        for char, freq in temp.items():
            arr.append((char, freq))
        
        # sort in ascending order 
        arr.sort(key=lambda tup: tup[1])
        print(arr)
        
        q = Queue(len(arr))
        
        for pair in arr:
            q.enqueue(pair)
        
        aux_q = Queue(len(arr))
        


In [61]:
huffman_code(["88888888889999999999yyyyyyyyhhhhhhhjjjjj..../321zx"])

[('/', 1), ('3', 1), ('2', 1), ('1', 1), ('z', 1), ('x', 1), ('.', 4), ('j', 5), ('h', 7), ('y', 8), ('8', 10), ('9', 10)]
('/', 1)
('3', 1)
('2', 1)
('1', 1)
('z', 1)
('x', 1)
('.', 4)
('j', 5)
('h', 7)
('y', 8)
('8', 10)
('9', 10)


Solution for Problem 4 - Maximum Flow/ Minimum Cut Problem